In [1]:
import json
from subprocess import Popen, PIPE
import shlex
import os
import jsonlines
from glob import glob
from tqdm.notebook import tqdm

In [2]:
input_paths = glob("/home/lhk/data/hp_search/*/*.json")

In [ ]:
for input_path in tqdm(input_paths, position=0):

    with open(input_path) as file:
        data = json.load(file)

    parent_dir, _ = os.path.split(input_path)
    parent_dir += "/"

    total = 0

    for entry in tqdm(data, position=1, leave=False):
        in_path = parent_dir+entry["result_filepath"]
        result_path, _ = os.path.split(in_path)

        parsed = {in_path:[]}
        for obj in jsonlines.open(in_path):
            total+= len(obj["model_completions"])
            parsed[in_path].append({"id":obj['id'], "model_completions": obj["model_completions"]})

        prepared_path = result_path + "/cc_eval_format.json"
        out_path = result_path + "/output.json"
        
        if os.path.exists(out_path):
            continue
        
        with open(prepared_path, "w") as file:
            json.dump(parsed, file)


        cmd = f"bazel run -c opt execution:solve_example -- --data_path=/home/lhk/data/ --input_path={prepared_path} --output_path={out_path}   --python3_path=/usr/bin/python3.10 --python3_library_paths=/usr/lib/python3.10"

        process = Popen(shlex.split(cmd), stdout=PIPE, stderr=PIPE)
        stdout, stderr = process.communicate()
        
        #stdout = stdout.decode()
        #stderr = stderr.decode()
        
        #print(stdout, stderr)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/180 [00:00<?, ?it/s]

  0%|          | 0/135 [00:00<?, ?it/s]

  0%|          | 0/180 [00:00<?, ?it/s]

In [ ]:
out_path

In [ ]:
if os.path.exists(out_path):
    print("there")

In [ ]:
for p in processes:
    p.wait()

In [ ]:
passed = 0
failed = 0

for filename in glob("/home/lhk/data/hp_test/*/*/output.json"):
    with open(filename) as file:
        data = json.load(file)
        
    results = [res['passed'] for res in data['results']]
    passed += sum(results)
    failed += len(results) - sum(results)